## CS4486 Assignment 2
### Chicago Taxi Dataset

In [24]:
import numpy as np
import torch 
import torchvision
import torch.nn as nn

## Using Multi-layer Perceptron with 6 hidden linear layers

- With feature reduction: only 9 features were chosen out of 49 features. See [Feature Reduction](###Feature-Reduction) section below.

In [25]:
# load full train dataset
data = np.load("train.npz")

In [26]:
# extract feature arrays
x = data['x']
y = data['y']
locations = data['locations']
times= data['times']

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [28]:

train_dataset = np.zeros((len(y), (3*3*8+2+1)))
for num_instance in range(len(y)):
  for time_index in range(8):
    for nbr_row_index in range(3):
      for nbr_col_index in range(3):
        train_dataset[num_instance][time_index*9 + nbr_row_index*3 + nbr_col_index] = x[num_instance][time_index][(2+nbr_row_index)*7 + (nbr_col_index+3)]
  train_dataset[num_instance][8*9] = locations[num_instance][0]
  train_dataset[num_instance][8*9+1] = locations[num_instance][1]
  train_dataset[num_instance][8*9+2] = times[num_instance]
#train_dataset = np.array([i, x[i].ravel()+[locations[i][0], locations[i][1], times[i]]] for i in range(len(y)))


### Feature Reduction

Feature reduction happens in that only data matrix at hour index 0, 1, 2, ..., 7 (i.e. for the i-th row x[i][0], x[i][1], x[i][2], ... x[i][7]), where each matrix **D** has items within **D**(2, 2) - **D**(4, 4) taken out and concatenated in the aforementioned order.  

```
Legend:    O - not taken
           X - taken
           
    0 1 2 3 4 5 6    
    _ _ _ _ _ _ _  
0 | O O O O O O O
1 | O O O O O O O
2 | O O X X X O O
3 | O O X X X O O
4 | O O X X X O O
5 | O O O O O O O
6 | O O O O O O O
```  

### Structure of ```train_dataset```  
Inside ```train_dataset```, each (num_instance)-th entry is a flattened 1D array consits of below:  
(1) An 1D array that is the concatenation of all 8 matrices described in the [Feature Reduction](###Feature-Reduction) section above.  
(2) the x, y coordinates of the (num_instance)-th entry locations[]   
(3) value of the (num_instance)-th entry in the times[] array

Therefore in total the dimension of train_dataset is Row=len(y) and Coloumn= [8(hours) * 9(3*3 matrix of neighbor for each hour) + 2(location-x, location-y) +1(times)] = 75.

In [29]:
print(train_dataset.shape)

(72000, 75)


In [30]:
# convert numpy array to tensor
x_tensor = torch.Tensor(train_dataset)
y_tensor = torch.Tensor(y)

# set up loader for training dataset
train_dataset_tensor  = torch.utils.data.TensorDataset(x_tensor, y_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset_tensor, batch_size=100, shuffle=True)


```input_size``` = 75 is the size of flattened array starting from ndim(1)  
Number of output units: ```num_classes``` = **1**  since we only want one regression value  

In [31]:
# define constants 
input_size = 75 # 75 input node
layer1_hidden_size = 38
layer2_hidden_size = 19
layer3_hidden_size = 10
layer4_hidden_size = 5
layer5_hidden_size = 3
layer6_hidden_size = 2
hidden_size = [layer1_hidden_size, layer2_hidden_size, layer3_hidden_size, layer4_hidden_size, layer5_hidden_size, layer6_hidden_size]
output_size = 1
learning_rate = 0.001


In [32]:
# NN with one 6 layer and Linear transformation
class NeuralNet(nn.Module):
  # hidden_size: list of hidden layer size
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        layer1_hidden_size = hidden_size[0]
        layer2_hidden_size = hidden_size[1]
        layer3_hidden_size = hidden_size[2]
        layer4_hidden_size = hidden_size[3]
        layer5_hidden_size = hidden_size[4]
        layer6_hidden_size = hidden_size[5]
        self.fc1 = nn.Linear(input_size, layer1_hidden_size) 
        self.fc2 = nn.Linear(layer1_hidden_size, layer2_hidden_size) 
        self.fc3 = nn.Linear(layer2_hidden_size, layer3_hidden_size) 
        self.fc4 = nn.Linear(layer3_hidden_size, layer4_hidden_size) 
        self.fc5 = nn.Linear(layer4_hidden_size, layer5_hidden_size) 
        self.fc6 = nn.Linear(layer5_hidden_size, layer6_hidden_size) 
        self.fc7 = nn.Linear(layer6_hidden_size, output_size)
        self.relu = nn.ReLU()
         
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)
        out = self.relu(out)
        out = self.fc5(out)
        out = self.relu(out)
        out = self.fc6(out)
        out = self.relu(out)
        out = self.fc7(out)
        return out

In [41]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [42]:
# the accuracy score
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [43]:
print("="*30 + "TRAINING" + "="*30)
total_step = len(train_loader)
num_epochs = 5
for epoch in range(num_epochs):
  for i, (x_data, y_data) in enumerate(train_loader):
    x_data = x_data.to(device)
    y_data = y_data.to(device)

    # forward pass
    outputs = model(x_data)
    loss = criterion(outputs, y_data)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 120 == 0:
      print('Epoch [ %d/%d ], Step [ %d/%d ], Loss is %.4f' %
            (epoch + 1, num_epochs, i+1, total_step, loss.item()))

==============================TRAINING==============================
Epoch [ 1/5 ], Step [ 120/720 ], Loss is 127.6679
Epoch [ 1/5 ], Step [ 240/720 ], Loss is 131.4054
Epoch [ 1/5 ], Step [ 360/720 ], Loss is 252.4255
Epoch [ 1/5 ], Step [ 480/720 ], Loss is 299.7838
Epoch [ 1/5 ], Step [ 600/720 ], Loss is 193.9176
Epoch [ 1/5 ], Step [ 720/720 ], Loss is 187.1312
Epoch [ 2/5 ], Step [ 120/720 ], Loss is 179.5204
Epoch [ 2/5 ], Step [ 240/720 ], Loss is 275.6231
Epoch [ 2/5 ], Step [ 360/720 ], Loss is 83.7387
Epoch [ 2/5 ], Step [ 480/720 ], Loss is 178.8599
Epoch [ 2/5 ], Step [ 600/720 ], Loss is 230.1450
Epoch [ 2/5 ], Step [ 720/720 ], Loss is 131.8691
Epoch [ 3/5 ], Step [ 120/720 ], Loss is 132.7662
Epoch [ 3/5 ], Step [ 240/720 ], Loss is 190.3132
Epoch [ 3/5 ], Step [ 360/720 ], Loss is 297.9509
Epoch [ 3/5 ], Step [ 480/720 ], Loss is 146.3314
Epoch [ 3/5 ], Step [ 600/720 ], Loss is 58.1541
Epoch [ 3/5 ], Step [ 720/720 ], Loss is 116.6117
Epoch [ 4/5 ], Step [ 120/720 ], 

In [38]:
print("="*30 + "Load the val set" + "="*30)
# Load the val set
val_data = np.load('val.npz')
val_x = val_data['x']
val_y = val_data['y']
val_times = val_data['times']
val_locations = val_data['locations']

# self-brewed val dataset
val_dataset = np.zeros((len(val_y), (3*3*8+2+1)))
for num_instance in range(len(val_y)):
  for time_index in range(8):
    for nbr_row_index in range(3):
      for nbr_col_index in range(3):
        val_dataset[num_instance][time_index*9 + nbr_row_index*3 + nbr_col_index] = val_x[num_instance][time_index][(2+nbr_row_index)*7 + (nbr_col_index+3)]
  val_dataset[num_instance][8*9] = val_locations[num_instance][0]
  val_dataset[num_instance][8*9+1] = val_locations[num_instance][1]
  val_dataset[num_instance][8*9+2] = val_times[num_instance]

print("val_dataset shape:",val_dataset.shape)


==============================Load the val set==============================
val_dataset shape: (18000, 75)


In [39]:
print("="*30 + "create validation set loader" + "="*30)

# create validation set loader
val_x_tensor = torch.Tensor(val_dataset)
val_y_tensor = torch.Tensor(val_y)

val_dataset_tensor  = torch.utils.data.TensorDataset(val_x_tensor, val_y_tensor)
val_loader = torch.utils.data.DataLoader(val_dataset_tensor, batch_size=100, shuffle=False)

==============================create validation set loader==============================


In [44]:
print("="*30 + "Validation set" + "="*30)

# Get accuracy score on validation set
# Turn off gradient because we don't need to update the model
with torch.no_grad():
  correct = 0
  total = 0
  for x_data, y_data in val_loader:
    x_data = x_data.to(device)
    y_data = y_data.to(device)
    outputs = model(x_data)
    total += x_data.size(0)
    correct += 1 if np.isclose(outputs, y_data, atol=1e-01)[0] == True else 0
  
  print('Accuracy score: %.4f' % (100.0 * correct / total,))

==============================Validation set==============================
Accuracy score: 0.2333
